In [2]:
import pandas as pd
import numpy as np
from pandas import ExcelWriter
import os

In [3]:
Working = pd.read_excel("Sales.xlsx")

In [4]:
Working  = Working[['Bill Print Invoice Number','GST Regn No of BP','Type','Posting Date','Customer/Vendor Name',
    'Item Name','Quantity','Inventory UoM','Chapter ID','GST Tax Rate','POS (Place Of Supply)','Row Total','CGST','SGST','IGST','Line Total with Tax Amount']]

In [5]:
Working.rename(columns={'Chapter ID':"HSN Code","Posting Date":"Document Date","GST Regn No of BP":"GST Number","Bill Print Invoice Number":"Bill Number",
    'POS (Place Of Supply)':'Place Of Supply',"Type":'Document Type',"Row Total":'Taxable Value',"Line Total with Tax Amount":'Total Amount',"Inventory UoM":"UQC"},inplace=True)

In [6]:
cod = [Working['Document Type']=="AR Invoice",Working['Document Type']=='A/R Credit Note',Working['Document Type']=='A/R Debit Note']
values = ['Invoice',"Credit Note",'Debit Note']
Working['Document Type']=np.select(cod,values)

# Prepare Data for Invoice value
Voucher_Amount = Working.pivot_table(values=['Total Amount'],index='Bill Number',aggfunc=sum)
Voucher_Amount = Voucher_Amount.reset_index()
Voucher_Amount.rename(columns={'Bill Number':'Invoice Number','Total Amount':"Document Amount"},inplace=True)


In [7]:
####################################################################
###################### Prepare ANX folder ##########################
####################################################################

path = os.getcwd()+ '\\Anx'

isexist = os.path.exists(path)

if not isexist:
    os.makedirs(path)
    

In [8]:
os.getcwd()

'c:\\Users\\Vishal Sharma\\Desktop\\Python\\GSTR1'

In [9]:
####################################################################
###############  Prepare B2B Data for CSV ##########################
####################################################################

#Seprate Data for B2B Data
B2B= Working[ (Working['Document Type'] == 'Invoice') & (Working['GST Number'] != np.nan)]
B2B.rename(columns={'GST Number':'GSTIN/UIN of Recipient','Customer/Vendor Name':'Receiver Name','Bill Number':"Invoice Number",'Document Date':'Invoice date',
    'GST Tax Rate':'Rate'},inplace= True)


# Add furhter columns
B2B['Reverse Charge'] = 'N'
B2B['Applicable % of Tax Rate'] = ''
B2B['Invoice Type'] = 'Regular'
B2B['E-Commerce GSTIN'] = ''
B2B['Cess Amount'] = 0

B2B= pd.merge(B2B,Voucher_Amount, how='inner',on=['Invoice Number'])
B2B.rename(columns={'Document Amount':'Invoice Value'},inplace=True)

B2B['Taxable Value'] = B2B['Taxable Value'].round(2)
B2B['Invoice Value'] = B2B['Invoice Value'].round(2)


t = B2B.pivot_table(values='Taxable Value',index=['GSTIN/UIN of Recipient','Invoice Number','Invoice date','Rate'],aggfunc=sum)

B2B= B2B[['GSTIN/UIN of Recipient','Receiver Name','Invoice Number','Invoice date','Invoice Value','Place Of Supply','Reverse Charge','Applicable % of Tax Rate',
    'Invoice Type', 'E-Commerce GSTIN', 'Rate','Cess Amount']]


B2B= B2B.drop_duplicates()
B2B = pd.merge(t,B2B,how='inner',on=['GSTIN/UIN of Recipient','Invoice Number','Invoice date','Rate'])
B2B= B2B.reset_index()
B2B = B2B[['GSTIN/UIN of Recipient','Receiver Name','Invoice Number','Invoice date','Invoice Value','Place Of Supply','Reverse Charge','Applicable % of Tax Rate',
    'Invoice Type', 'E-Commerce GSTIN', 'Rate','Taxable Value','Cess Amount']]


B2B.to_csv( path+'\B2B.csv',index=False)


C:\Users\Vishal Sharma\AppData\Local\Temp\ipykernel_21688\1991817565.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  B2B.rename(columns={'GST Number':'GSTIN/UIN of Recipient','Customer/Vendor Name':'Receiver Name','Bill Number':"Invoice Number",'Document Date':'Invoice date',
C:\Users\Vishal Sharma\AppData\Local\Temp\ipykernel_21688\1991817565.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  B2B['Reverse Charge'] = 'N'
C:\Users\Vishal Sharma\AppData\Local\Temp\ipykernel_21688\1991817565.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy

In [10]:
####################################################################
###############  Prepare CNDR Data for CSV ##########################
####################################################################

#Seprate Data for B2B Data
CDNR= Working[ (Working['Document Type']=='Credit Note') | (Working['Document Type']=='Debit Note') ]
CDNR = CDNR.rename(columns={'GST Number':'GSTIN/UIN of Recipient','Customer/Vendor Name':'Receiver Name',
    'Bill Number':'Note Number','Document Date':'Note Date','Document Type':'Note Type','GST Tax Rate':"Rate",
    })

CDNR = CDNR[['GSTIN/UIN of Recipient','Receiver Name','Note Number','Note Date','Note Type',
    'Place Of Supply','Rate','Taxable Value']]


cod =[ CDNR['Note Type']=='Credit Note',CDNR['Note Type']=='Debit Note']
values = ['C','D']
CDNR['Note Type']=np.select(cod,values)

Voucher_Amount.rename(columns={'Invoice Number':'Note Number'},inplace=True)

CDNR=  pd.merge(CDNR,Voucher_Amount,how='inner',on=['Note Number'])
CDNR.rename(columns={'Document Amount':'Note Value'},inplace=True)
CDNR['Reverse Charge'] = 'N'
CDNR['Note Supply Type'] = 'Regular'
CDNR['Applicable % of Tax Rate'] = ''
CDNR['Cess Amount'] = 0

CDNR = CDNR[['GSTIN/UIN of Recipient','Receiver Name','Note Number','Note Date','Note Type',
    'Place Of Supply','Reverse Charge','Note Supply Type','Note Value','Applicable % of Tax Rate'
    ,'Rate','Taxable Value','Cess Amount']]

CDNR['Taxable Value'] =CDNR['Taxable Value'].abs().round(2)
CDNR['Note Value'] =CDNR['Note Value'].abs().round(2)
CDNR.to_csv(path+'\cdnr.csv',index=False)


In [11]:
####################################################################
###############  Prepare HSN Data for CSV ##########################
####################################################################

#Seprate Data for HSN Data
HSN = Working[['HSN Code','UQC','Quantity','Total Amount', 'Taxable Value','IGST', 'CGST','SGST','GST Tax Rate']]



# Change Columns Name
HSN.rename(columns={'HSN Code':'HSN','Quantity':'Total Quantity','Total Amount':'Total Value',
    'IGST':'Integrated Tax Amount','CGST':'Central Tax Amount','SGST':'State/UT Tax Amount',
    'GST Tax Rate':'Rate'},inplace=True)

HSN['Description']= HSN['HSN']

HSN = HSN[['HSN','Description','UQC', 'Total Quantity', 'Total Value', 'Taxable Value',
       'Integrated Tax Amount', 'Central Tax Amount', 'State/UT Tax Amount',
       'Rate' ]]



cod = [HSN['UQC']=='BOX',HSN['UQC']=='KGS',HSN['UQC']=='CLD',HSN['UQC']=='NOS']
values= ['BOX-BOX','KGS-KILOGRAMS','BOX-BOX','NOS-NUMBERS']
HSN['UQC']= np.select(cod,values)


HSN= HSN.pivot_table(values=['Total Quantity', 'Total Value',
       'Taxable Value', 'Integrated Tax Amount', 'Central Tax Amount',
       'State/UT Tax Amount'],index=['HSN', 'Description', 'UQC', 'Rate'],aggfunc=sum)

HSN = HSN.reset_index()

HSN['Cess Amount'] =0

HSN = HSN[['HSN', 'Description', 'UQC', 'Total Quantity', 'Total Value','Taxable Value', 'Integrated Tax Amount',
       'Central Tax Amount','State/UT Tax Amount', 'Cess Amount','Rate']]

for i in ['Total Quantity', 'Total Value','Taxable Value', 'Integrated Tax Amount',
       'Central Tax Amount','State/UT Tax Amount', 'Cess Amount']:
       HSN[i]=HSN[i].round(2)


for i in range(len(HSN)):
    HSN.iloc[i,0] = HSN.iloc[i,0][:2] + HSN.iloc[i,0][3:5]  + HSN.iloc[i,0][6:] 


HSN.to_csv(path+'\HSN.csv',index=False)

C:\Users\Vishal Sharma\AppData\Local\Temp\ipykernel_21688\2111549316.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HSN.rename(columns={'HSN Code':'HSN','Quantity':'Total Quantity','Total Amount':'Total Value',
C:\Users\Vishal Sharma\AppData\Local\Temp\ipykernel_21688\2111549316.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HSN['Description']= HSN['HSN']
